In [109]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'
}


def get_detail_info(person_node):
    url = "https://cmn.nimh.nih.gov/" + person_node
    response = requests.get(url, headers=headers, timeout=10)
    soup = BeautifulSoup(response.text, 'html.parser')
    title = soup.find('span', class_='field--name-title').text.strip()

    image_url = soup.find('img', loading="lazy")['src']
    image_name = image_url.split("/")[-1]

    img_response = requests.get("https://cmn.nimh.nih.gov/" + image_url)
    with open(image_name, 'wb') as f:
        f.write(img_response.content)
        
    # Extract the job title
    position = ""
    job_title_span = soup.find('div', class_='field--name-field-job-title')
    if job_title_span:
        job_title_span = job_title_span.find('div', class_='field__item')
        position = job_title_span.text.strip() if job_title_span else ""

    # Extract all social links
    social_links = {}
    links_list = soup.find('div', class_='field--name-field-links')
    if links_list:
        links_list = links_list.find_all('a')
    
        for link in links_list:
            social_links[link.text.strip()] = link['href']

    # Get the current time
    now = datetime.now()
    
    # Format the time
    formatted_time = now.strftime("%Y-%m-%d %H:%M")

    data =  {
        "Title": title,
        "Position": position,
        "Image": image_name,
        **social_links,
        "Slug": person_node.split("/")[-1],
        "Date": formatted_time
    }

    file_name = title.replace(" ", "")
    with open(f'{file_name}.md', 'w') as file:
        for key, value in data.items():
            file.write(f"{key}: {value}\n")


In [96]:
base_url = 'https://cmn.nimh.nih.gov/people'
response = requests.get(base_url, headers = headers, timeout=10)
soup = BeautifulSoup(response.text, 'html.parser')

members = soup.find_all('a', rel='bookmark')

In [122]:
for i, m in enumerate(members):
    print(i, m['href'])

0 /node/7
1 /node/97
2 /node/99
3 /node/53
4 /node/12
5 /node/50
6 /node/8
7 /node/11
8 /node/95
9 /node/80
10 /node/83
11 /node/9
12 /node/52
13 /node/51
14 /node/78
15 /node/46
16 /node/55
17 /node/60
18 /node/71
19 /node/82
20 /node/21
21 /node/18
22 /node/25
23 /node/57
24 /node/24
25 /node/23
26 /node/22


In [126]:
for member in members[22:]:
    member_url = member['href']
    print(member_url)
    get_detail_info(member_url)
    

/node/25
/node/57
/node/24
/node/23
/node/22
